In [1]:
"""
Gemma-3-1B Model Architecture Inspector
Extracts detailed model specifications for probing experiments
"""

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import torch

def inspect_gemma_model():
    model_name = "google/gemma-3-1b-it"
    
    print(f"🔍 Inspecting {model_name}")
    print("=" * 60)
    
    # Load configuration
    config = AutoConfig.from_pretrained(model_name)
    print("\n📋 MODEL CONFIGURATION:")
    print(f"  Model type: {config.model_type}")
    print(f"  Architecture: {config.architectures}")
    print(f"  Hidden size: {config.hidden_size}")
    print(f"  Number of layers: {config.num_hidden_layers}")
    print(f"  Number of attention heads: {config.num_attention_heads}")
    print(f"  Intermediate size: {config.intermediate_size}")
    print(f"  Vocabulary size: {config.vocab_size}")
    print(f"  Max position embeddings: {config.max_position_embeddings}")
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print(f"\n🔤 TOKENIZER DETAILS:")
    print(f"  Tokenizer type: {type(tokenizer).__name__}")
    print(f"  Vocabulary size: {len(tokenizer)}")
    print(f"  Special tokens: {tokenizer.special_tokens_map}")
    
    # Test tokenization of key phrases
    test_phrases = [
        "I think the emotional state of this user is",
        "is",
        " is",
        "state is"
    ]
    
    print(f"\n🧪 TOKENIZATION TESTS:")
    for phrase in test_phrases:
        tokens = tokenizer.tokenize(phrase)
        token_ids = tokenizer.encode(phrase, add_special_tokens=False)
        print(f"  '{phrase}' → {tokens} → {token_ids}")
    
    # Load model and inspect architecture
    print(f"\n🏗️  LOADING MODEL...")
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_name, 
            output_hidden_states=True,
            torch_dtype=torch.float16,  # Use half precision to save memory
            device_map="auto" if torch.cuda.is_available() else "cpu"
        )
        model.eval()
        
        print(f"  ✅ Model loaded successfully")
        print(f"  Device: {next(model.parameters()).device}")
        print(f"  Data type: {next(model.parameters()).dtype}")
        
        # Get model size info
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        
        print(f"\n📊 MODEL SIZE:")
        print(f"  Total parameters: {total_params:,}")
        print(f"  Trainable parameters: {trainable_params:,}")
        print(f"  Model size: ~{total_params * 2 / 1e9:.2f} GB (fp16)")
        
        # Test hidden state extraction
        test_text = "I think the emotional state of this user is"
        print(f"\n🧠 HIDDEN STATE EXTRACTION TEST:")
        print(f"  Test input: '{test_text}'")
        
        inputs = tokenizer(test_text, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        print(f"  Input shape: {inputs['input_ids'].shape}")
        print(f"  Input tokens: {inputs['input_ids'][0].tolist()}")
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        hidden_states = outputs.hidden_states
        print(f"  Number of layers (including embedding): {len(hidden_states)}")
        
        for i, layer_states in enumerate(hidden_states):
            print(f"  Layer {i}: {layer_states.shape} → Last token vector: {layer_states[0, -1, :].shape}")
        
        # Specific details for your probing experiment
        print(f"\n🎯 PROBING EXPERIMENT DETAILS:")
        print(f"  Hidden dimension for LogisticRegression input: {config.hidden_size}")
        print(f"  Number of probe layers to train: {len(hidden_states)}")
        print(f"  Expected feature vector shape per sample: ({config.hidden_size},)")
        print(f"  Memory per hidden state vector: {config.hidden_size * 4} bytes (fp32)")
        print(f"  Memory per sample (all layers): {len(hidden_states) * config.hidden_size * 4} bytes")
        
        # Estimate memory for your dataset
        n_samples = 440  # From your dataset
        total_memory_mb = (n_samples * len(hidden_states) * config.hidden_size * 4) / (1024 * 1024)
        print(f"  Estimated memory for 440 samples: {total_memory_mb:.2f} MB")
        
        return {
            'hidden_size': config.hidden_size,
            'num_layers': len(hidden_states),
            'vocab_size': config.vocab_size,
            'model_config': config,
            'tokenizer': tokenizer
        }
        
    except Exception as e:
        print(f"  ❌ Error loading model: {e}")
        print(f"  Falling back to config-only inspection")
        
        return {
            'hidden_size': config.hidden_size,
            'num_layers': config.num_hidden_layers + 1,  # +1 for embedding layer
            'vocab_size': config.vocab_size,
            'model_config': config,
            'tokenizer': tokenizer
        }

def get_probe_input_specs():
    """Returns the exact specifications for your logistic regression probes"""
    details = inspect_gemma_model()
    
    print(f"\n" + "="*60)
    print("📋 LOGISTIC REGRESSION PROBE SPECIFICATIONS")
    print("="*60)
    print(f"Input feature dimension: {details['hidden_size']}")
    print(f"Number of probes to train: {details['num_layers']}")
    print(f"Output classes: 4 (Happy, Sad, Angry, Neutral)")
    print(f"Probe type: LogisticRegression(multi_class='ovr')")
    print(f"Expected X.shape per layer: (n_samples, {details['hidden_size']})")
    print(f"Expected y.shape: (n_samples,)")
    
    return details

if __name__ == "__main__":
    model_details = get_probe_input_specs()

/Users/chinmaysultanpuri/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔍 Inspecting google/gemma-3-1b-it

📋 MODEL CONFIGURATION:
  Model type: gemma3_text
  Architecture: ['Gemma3ForCausalLM']
  Hidden size: 1152
  Number of layers: 26
  Number of attention heads: 4
  Intermediate size: 6912
  Vocabulary size: 262144
  Max position embeddings: 32768

🔤 TOKENIZER DETAILS:
  Tokenizer type: GemmaTokenizerFast
  Vocabulary size: 262145
  Special tokens: {'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'boi_token': '<start_of_image>', 'eoi_token': '<end_of_image>', 'image_token': '<image_soft_token>'}

🧪 TOKENIZATION TESTS:
  'I think the emotional state of this user is' → ['I', '▁think', '▁the', '▁emotional', '▁state', '▁of', '▁this', '▁user', '▁is'] → [236777, 1751, 506, 13690, 1883, 529, 672, 2430, 563]
  'is' → ['is'] → [511]
  ' is' → ['▁is'] → [563]
  'state is' → ['state', '▁is'] → [3255, 563]

🏗️  LOADING MODEL...


The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  ✅ Model loaded successfully
  Device: cpu
  Data type: torch.float16

📊 MODEL SIZE:
  Total parameters: 999,885,952
  Trainable parameters: 999,885,952
  Model size: ~2.00 GB (fp16)

🧠 HIDDEN STATE EXTRACTION TEST:
  Test input: 'I think the emotional state of this user is'
  Input shape: torch.Size([1, 10])
  Input tokens: [2, 236777, 1751, 506, 13690, 1883, 529, 672, 2430, 563]
  Number of layers (including embedding): 27
  Layer 0: torch.Size([1, 10, 1152]) → Last token vector: torch.Size([1152])
  Layer 1: torch.Size([1, 10, 1152]) → Last token vector: torch.Size([1152])
  Layer 2: torch.Size([1, 10, 1152]) → Last token vector: torch.Size([1152])
  Layer 3: torch.Size([1, 10, 1152]) → Last token vector: torch.Size([1152])
  Layer 4: torch.Size([1, 10, 1152]) → Last token vector: torch.Size([1152])
  Layer 5: torch.Size([1, 10, 1152]) → Last token vector: torch.Size([1152])
  Layer 6: torch.Size([1, 10, 1152]) → Last token vector: torch.Size([1152])
  Layer 7: torch.Size([1, 10, 1